In [11]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [12]:
### casi todas lo hacen así, puede que otra no porque percibe las cookies como otra capa
driver = webdriver.Chrome()
url_wunder = "https://www.guias.travel/tour/"
driver.get(url_wunder)
# colocar prints
driver.maximize_window()
driver.implicitly_wait(5) ### es como un sleep, pero aquí va a intentar lo siguiente durante el tiempo que le digo
#aceptamos las cookies
### para las cookies
### el tiempo de python puede ser diferente al navegador

In [13]:
try: 
    driver.find_element("css selector", '#search-form-input').click()
except:
    print('No encuentro el botón')

driver.find_element('css selector', '#search-form-input').send_keys('florencia', Keys.ENTER)

In [14]:
sopa_excursiones = BeautifulSoup(driver.page_source)
sopa_excursiones

body > div.container.margin_60.tours_archive_container > div > div.col-lg-9.col-md-8 > div.tour-list > a:nth-child(1) > div > div

<html lang="es-ES"><head><meta content="A/kargTFyk8MR5ueravczef/wIlTkbVk1qXQesp39nV+xNECPdLBVeYffxrM8TmZT6RArWGQVCJ0LRivD7glcAUAAACQeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZzIiLCJleHBpcnkiOjE3NDIzNDIzOTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9" http-equiv="origin-trial"/>
<title>Guias.travel - 32 actividades y experiencias encontradas</title>
<meta content="Type=text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="https://www.guias.travel/wp-content/themes/citytours/img/logo_og.png" property="og:image"/>
<style media="all">@charset "UTF-8";.wp-block-archives-dropdown label{display:block}.wp-block-avatar{box-sizing:border-box}.wp-block-avatar.aligncenter{text-align:center}.wp-block-audio{box-sizing:border-box}.wp-block-audio figcaption{margin-top:.5em;margin-bottom:1em}.wp-block-audio audio{width:100%;min-width:300px}.wp-block-button__link{cursor:pointer;display:inline-block;text-align:cente

In [ ]:
body > div.container.margin_60.tours_archive_container > div > div.col-lg-9.col-md-8